In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ri_neg = pd.read_csv('iTunes-Amazon/experiments-results/exp10/ri_neg_couples.csv')
ri_pos = pd.read_csv('iTunes-Amazon/experiments-results/exp10/ri_pos_couples.csv')

In [ ]:
attributes = list(ri_neg)
left_att_sum_neg = []
right_att_sum_neg = []
for i,att in enumerate(attributes):
    if i%2 == 0:
        left_att_sum_neg.append(round(ri_neg[att].sum()))
    else:
        right_att_sum_neg.append(round(ri_neg[att].sum()))

In [ ]:
left_att_sum_pos = []
right_att_sum_pos = []
for i,att in enumerate(attributes):
    if i%2 == 0:
        left_att_sum_pos.append(round(ri_pos[att].sum()))
    else:
        right_att_sum_pos.append(round(ri_pos[att].sum()))

In [ ]:
labels = ['Art-Gen', 'Art-Song', 'CR-Genr', 'Art-Alb', 'Art-CR','Gen-Rel','Alb-Gen','Tim-Alb']

In [ ]:
labels = ['Art-Gen', 'Art-Song', 'CR-Genr', 'Art-Alb', 'Art-CR','Gen-Rel','Alb-Gen','Tim-Alb']
men_means = left_att_sum_neg
women_means = right_att_sum_neg

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, men_means, width, label='l_Att')
rects2 = ax.bar(x + width/2, women_means, width, label='r_Att')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ri')
ax.set_title('Ri for negative samples')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()
plt.savefig('neg_barchart_ri.png')
plt.show()

In [ ]:
labels = ['Art-Gen', 'Art-Song', 'CR-Genr', 'Art-Alb', 'Art-CR','Gen-Rel','Alb-Gen','Tim-Alb']
men_means = left_att_sum_pos
women_means = right_att_sum_pos

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, men_means, width, label='l_Att')
rects2 = ax.bar(x + width/2, women_means, width, label='r_Att')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ri')
ax.set_title('Ri for positive samples')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()
plt.savefig('pos_barchart_ri.png')
plt.show()

In [2]:
n = 10

In [3]:
t = n,